# import lib

In [1]:
import cv2
import numpy as np
import os
import time
import pandas as pd
import mediapipe as mp
import threading
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from tensorflow import keras

from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


save history

In [2]:
# point_landmark_toget = [0, 11 ,12 ,13, 14,15 ,16, 19, 20, 23, 24]
# hàm để lấy 24 tọa độ khung xương
def get_frame_landmarks(results):
    size_landmarks = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[:23]]).flatten() if results.pose_landmarks else np.zeros(4*23)
    return size_landmarks

# hàm để dự đoán khung xương
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# hàm để chuyển đổi tọa độ khung xương và convert sang dạng phù hợp để vẽ lên ảnh
def get_scaled_landmarks(landmarks, dimenson):
    landmarks_2d = []
    if dimenson == '2d':
        for landmark in landmarks:
            x, y = int(landmark.x*1280), int(landmark.y*720)
            landmarks_2d.append([x, y])
        return landmarks_2d

# hàm hiển thị fps
def show_fps(image, prev_frame_time):
    new_frame_time = time.time()
    fps = int(1/(new_frame_time-prev_frame_time))
    cv2.putText(image, f"fps: {fps}", (1000, 700), cv2.FONT_HERSHEY_SIMPLEX, 2, (100, 255, 0), 2, cv2.LINE_AA)
    return new_frame_time
        
# hàm hiển thị vẽ khung xương
def draw_landmark_on_image( results, image):
    lmks = results.pose_landmarks.landmark
    
    pose_landmarks = [lmks[0], lmks[1], lmks[2], lmks[3], lmks[4], lmks[5], lmks[6], lmks[7], lmks[8], lmks[9], lmks[10], lmks[11], lmks[12], lmks[13], lmks[14], lmks[15], lmks[16], lmks[17], lmks[18], lmks[19], lmks[20], lmks[21], lmks[22], lmks[23], lmks[24]] 
    pose_landmarks = get_scaled_landmarks(pose_landmarks, '2d')
  
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[4]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[1]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[4]), tuple(pose_landmarks[6]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[6]), tuple(pose_landmarks[8]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[1]), tuple(pose_landmarks[3]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[3]), tuple(pose_landmarks[7]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[9]), tuple(pose_landmarks[10]), (255, 255, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[12]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[23]), (255, 205, 0), 2)
    
    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[23]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[12]), (0, 144, 255), 2)
        
    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[16]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[12]), (0, 144, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[22]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[20]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[18]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[18]), tuple(pose_landmarks[20]), (0, 144, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[21]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[19]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[17]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[19]), tuple(pose_landmarks[17]), (255, 205, 0), 2)
    
    
    for lm in pose_landmarks:
        
        cv2.circle(image, (int(lm[0]), int(lm[1])), 5, (255,255,255), 4)
        cv2.circle(image, (int(lm[0]), int(lm[1])), 4, (255,255,141), -1)

draw skeleton on image

In [ ]:
# Khởi tạo thư viện mediapipe và dự đoán trên 1 ảnh bất kỳ
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
mp_drawing = mp.solutions.drawing_utils

frame = cv2.imread("fpt_1.png")
frame = cv2.resize(frame,(1300,750)) 
frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

results = pose.process(frameRGB)
if results.pose_landmarks:
    # Ghi nhận thông số khung xương
    # Vẽ khung xương lên ảnh
    draw_landmark_on_image( results, frame)

    lm = get_frame_landmarks(results)

    print(lm)
cv2.imshow("image", frame)
cv2.imwrite("skeleton_fpt.png", frame)
# cv2.waitKey(1000)

# make data

get skeleton from video and save them into csv file
trích xuất khung xương và tạo data set để huấn luyện mô hình

In [ ]:
path_data = './data/'
path_video = os.path.join(path_data,'train')
path_csv = os.path.join(path_data,'landmark','train')
name_actions = ['cheat','non_cheat']

# Khởi tạo thư viện mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
mp_drawing = mp.solutions.drawing_utils

prev_frame_time = 0

In [ ]:
for name_action in name_actions:
    direction_path = os.path.join(path_video,name_action)
    lm_list = []
    frame_num = 0
    
    for id_sequence, video_name in enumerate(os.listdir(direction_path)):
        video_path = os.path.join(direction_path, video_name)
        cap = cv2.VideoCapture(video_path)
        
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Video end")
                break

            frame_num += 1
            print(frame_num)
            
            frame = cv2.resize(frame,(1300,750)) 
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(frameRGB)

            if results.pose_landmarks:
                # Ghi nhận thông số khung xương
                # Vẽ khung xương lên ảnh
                draw_landmark_on_image( results, frame)
                
                lm = get_frame_landmarks(results)
                
                lm_list.append(lm)
                print(lm)
#             prev_frame_time = show_fps(frame, prev_frame_time)
#             cv2.imshow("image", frame)
#             if cv2.waitKey(1) == ord('q'):
#                 break
  
#         cap.release()
        
    df  = pd.DataFrame(lm_list)
    
#     name_history = 'cheat_B.txt'
#     csv_per_action = os.path.join(path_csv,name_history)
#     df.to_csv(csv_per_action)
    
    name_history = name_action +'.csv'
    csv_per_action = os.path.join(path_csv,name_history)
    with open(csv_per_action, mode='w') as f:
        df.to_csv(f)

In [ ]:
path_data = './data/'
path_video = os.path.join(path_data,'test')
path_csv = os.path.join(path_data,'landmark','test')
name_actions = ['cheat','non_cheat']

# Khởi tạo thư viện mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
mp_drawing = mp.solutions.drawing_utils

prev_frame_time = 0

for name_action in name_actions:
    direction_path = os.path.join(path_video,name_action)
    lm_list = []
    frame_num = 0
    
    for id_sequence, video_name in enumerate(os.listdir(direction_path)):
        video_path = os.path.join(direction_path, video_name)
        cap = cv2.VideoCapture(video_path)
        
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Video end")
                break

            frame_num += 1
            print(frame_num)
            
            frame = cv2.resize(frame,(1300,750)) 
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frameRGB)

            if results.pose_landmarks:
                # Ghi nhận thông số khung xương
                # Vẽ khung xương lên ảnh
#                 draw_landmark_on_image( results, frame)
                
                lm = get_frame_landmarks(results)
                
                lm_list.append(lm)
                print(lm)
#             prev_frame_time = show_fps(frame, prev_frame_time)
#             cv2.imshow("image", frame)
#             if cv2.waitKey(1) == ord('q'):
#                 break
  
#         cap.release()
        
    df  = pd.DataFrame(lm_list)
    
#     name_history = 'cheat_B.txt'
#     csv_per_action = os.path.join(path_csv,name_history)
#     df.to_csv(csv_per_action)
    
    name_history = name_action +'.csv'
    csv_per_action = os.path.join(path_csv,name_history)
    with open(csv_per_action, mode='w') as f:
        df.to_csv(f)

# Train model 

In [3]:
# load file csv

#load dataset cho việc huấn luyện mô hình
cheat = pd.read_csv("./data/New folder/train/cheat.csv")
non_cheat = pd.read_csv("./data/New folder/train/non_cheat.csv")

X = []
y = []
no_of_timesteps = 10

dataset = cheat.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(1)
                        
dataset = non_cheat.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(0)
    
    
# test
cheat = pd.read_csv("./data/New folder/test/cheat.csv")
non_cheat = pd.read_csv("./data/New folder/test/non_cheat.csv")

X_test = []
y_test = []
no_of_timesteps = 10

dataset = cheat.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X_test.append(dataset[i-no_of_timesteps:i,:])
    y_test.append(1)
                        
dataset = non_cheat.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X_test.append(dataset[i-no_of_timesteps:i,:])
    y_test.append(0)

In [ ]:
X_train, y_train = np.array(X), np.array(y)
X_test, y_test = np.array(X_test), np.array(y_test)
X_val, y_val = X_test, y_test 

In [ ]:
## using LSTM architecture for training

def LSTM():
    inputs = tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    layer = keras.layers.LSTM(
        32, activation=tf.nn.relu6, return_sequences=True)(inputs)
    layer = keras.layers.Dropout(0.2)(layer)
    layer = keras.layers.LSTM(32, activation=tf.nn.relu6)(layer)
    layer = keras.layers.Dropout(0.2)(layer)
    layer = keras.layers.Dense(16, activation=tf.nn.relu6)(layer)
    layer = keras.layers.Dropout(0.2)(layer)

    outputs = keras.layers.Dense(units = 1, activation="sigmoid")(layer)

    model = keras.Model(inputs, outputs)

    model.summary()
    model.compile(optimizer="adam", metrics = ['accuracy'], loss = "binary_crossentropy")

    return model



def shalowCNN():
    inputs = tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    layer = keras.layers.Dense(16, activation=tf.nn.relu6)(inputs)
#     layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
#     layer = keras.layers.Dense(128, activation=tf.nn.relu6)(layer)
    layer = keras.layers.Flatten()(layer)
    layer = keras.layers.Dropout(0.2)(layer)
    outputs = keras.layers.Dense(units = 1, activation="sigmoid")(layer)

    model = keras.Model(inputs, outputs)

    model.summary()
    model.compile(optimizer="adam", metrics = ['accuracy'], loss = "binary_crossentropy")

    return model

def plot_confusion_matrix(plot_confusion_matrix_path,cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
        """Plots the confusion matrix."""
        if normalize:
          cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
          print("Normalized confusion matrix")
        else:
          print('Confusion matrix, without normalization')

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=55)
        plt.yticks(tick_marks, classes)
        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
          plt.text(j, i, format(cm[i, j], fmt),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        fig.savefig(plot_confusion_matrix_path)
        plt.close()
        

model = shalowCNN()

figure_data_path = './figures/shalowCNN/3'

checkpoint_path = os.path.join(figure_data_path,"weights.best.hdf5")
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                          patience = 10)
time_start = time.time()
history = model.fit(X_train,y_train ,
          batch_size=64,
          epochs = 500,
          validation_data=(X_val, y_val),
          callbacks=[checkpoint,earlystopping])

time_end = time.time()
model.save("model.h5")

model.load_weights(checkpoint_path)

loss_test,accuracy_test = model.evaluate(X_test, y_test)
print('LOSS TEST: ', loss_test)
print("ACCURACY TEST: ", accuracy_test)


loss_train, accuracy_train = model.evaluate(X_train, y_train)
print('LOSS TRAIN: ', loss_train)
print("ACCURACY TRAIN: ", accuracy_train)


data_eval = 'LOSS TEST: '+ str(loss_test) + ' \n ACCURACY TEST: '+ str(accuracy_test) + '\n' +'LOSS TRAIN: '+ str(loss_train) +'\n ACCURACY TRAIN: '+ str(accuracy_train) +"\n time : "+str((time_end - time_start)/60)

hist_df = pd.DataFrame(history.history)
name_history = 'history.csv'
path_history = os.path.join(figure_data_path , name_history)
with open(path_history, mode='w') as f:
    hist_df.to_csv(f)

# write EVAUATION into txt
Name_f = 'EVAUATION.txt'
# f = open('./figure/CNN/train_1st/EVAUATION_1st.txt','w+')

path_s= os.path.join(figure_data_path , Name_f)

with open(path_s, mode='w') as f:
    f.writelines(data_eval)
f.close()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(os.path.join(figure_data_path , 'acc_his.png'))
plt.close()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig(os.path.join(figure_data_path , 'loss_his.png'))
plt.close()


# test model

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import tensorflow as tf


def get_frame_landmarks(results):
    size_landmarks = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[:23]]).flatten() if results.pose_landmarks else np.zeros(4*23)
    return size_landmarks

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def get_scaled_landmarks(landmarks, dimenson):
    landmarks_2d = []
    if dimenson == '2d':
        for landmark in landmarks:
            x, y = int(landmark.x*1280), int(landmark.y*720)
            landmarks_2d.append([x, y])
        return landmarks_2d

def draw_landmark_on_image( results, image):
    lmks = results.pose_landmarks.landmark
    
    pose_landmarks = [lmks[0], lmks[1], lmks[2], lmks[3], lmks[4], lmks[5], lmks[6], lmks[7], lmks[8], lmks[9], lmks[10], lmks[11], lmks[12], lmks[13], lmks[14], lmks[15], lmks[16], lmks[17], lmks[18], lmks[19], lmks[20], lmks[21], lmks[22], lmks[23], lmks[24]] 
    pose_landmarks = get_scaled_landmarks(pose_landmarks, '2d')
  
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[4]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[1]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[4]), tuple(pose_landmarks[6]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[6]), tuple(pose_landmarks[8]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[1]), tuple(pose_landmarks[3]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[3]), tuple(pose_landmarks[7]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[9]), tuple(pose_landmarks[10]), (255, 255, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[12]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[23]), (255, 205, 0), 2)
    
    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[23]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[12]), (0, 144, 255), 2)
        
    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[16]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[12]), (0, 144, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[22]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[20]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[18]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[18]), tuple(pose_landmarks[20]), (0, 144, 255), 2)
    
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[21]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[19]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[17]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[19]), tuple(pose_landmarks[17]), (255, 205, 0), 2)
    
    
    for lm in pose_landmarks:
        
        cv2.circle(image, (int(lm[0]), int(lm[1])), 5, (255,255,255), 4)
        cv2.circle(image, (int(lm[0]), int(lm[1])), 4, (255,255,141), -1)

        

def show_fps(image, prev_frame_time):
    new_frame_time = time.time()
    fps = int(1/(new_frame_time-prev_frame_time))
    cv2.putText(image, f"fps: {fps}", (1000, 700), cv2.FONT_HERSHEY_SIMPLEX, 2, (100, 255, 0), 2, cv2.LINE_AA)
    return new_frame_time

def draw_class_on_image(label, img):
    
    fontColor = (255, 255 , 0)
    if label == "non_cheat":
        fontColor = (255, 255 , 0)
    elif label == "cheat":
        fontColor = (0, 69, 255)
        cv2.rectangle(img=img,
            pt1=(20, 20),
            pt2=(1260,700),
            color=(0, 144, 255),
            thickness=3,
            lineType=2,
            shift=0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (950, 550)
    fontScale = 1.5
    
    thickness = 3
    lineType = 2
    cv2.putText(img, label,
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                thickness,
                lineType)
    return img


def detect(model, lm_list):
    global label
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    print(lm_list.shape)
    results = model.predict(lm_list)
    print(results)
    if results[0][0] > 0.5:
        label = "cheat"
    else:
        label = "non_cheat"
    return label



In [ ]:
label = "Warmup...."
n_time_steps = 10
lm_list = []

mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

# model = tf.keras.models.load_model("./model.h5")
model.load_weights("./figures/shalowCNN/2/weights.best.hdf5")
# cap = cv2.VideoCapture('http://192.168.79.14:8080/video')
cap = cv2.VideoCapture("test_non.mp4")

# Khởi tạo thư viện mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

prev_frame_time = 0


i = 0
warmup_frames = 0
lm_list = []
while True:

    success, frame = cap.read()  
    
    image, results = mediapipe_detection(frame, pose)
    
    frame = cv2.resize(frame,(1280,720))
    
    i = i+1
    if i > warmup_frames:
        print("Start detect....")
        
        if results.pose_landmarks:
                # Ghi nhận thông số khung xương
                # Vẽ khung xương lên ảnh
                draw_landmark_on_image( results, frame)

                lm = get_frame_landmarks(results)

                
                lm_list.append(lm)
                print(lm)
        
            
                if len(lm_list) == n_time_steps:
                    t1 = threading.Thread(target=detect, args=(model, lm_list,))
                    t1.start()
                    lm_list = []

                print(lm)
            
    frame = draw_class_on_image(label, frame)
    prev_frame_time = show_fps(frame, prev_frame_time)
    cv2.imshow("image", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# expand web cam and mobie cam (detect on face and body)

# pose detect

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import tensorflow as tf

def get_frame_landmarks(results):
    size_landmarks = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[:23]]).flatten() if results.pose_landmarks else np.zeros(4*23)
    return size_landmarks

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def get_scaled_landmarks(landmarks, dimenson):
    landmarks_2d = []
    if dimenson == '2d':
        for landmark in landmarks:
            x, y = int(landmark.x*1280), int(landmark.y*720)
            landmarks_2d.append([x, y])
        return landmarks_2d

def draw_landmark_on_image( results, image):
    lmks = results.pose_landmarks.landmark
    pose_landmarks = [lmks[0], lmks[11], lmks[12], lmks[13], lmks[14], lmks[15], lmks[16], lmks[23], lmks[24], lmks[19], lmks[20]] 
    pose_landmarks = get_scaled_landmarks(pose_landmarks, '2d')
    
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[4]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[0]), tuple(pose_landmarks[1]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[4]), tuple(pose_landmarks[6]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[6]), tuple(pose_landmarks[8]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[1]), tuple(pose_landmarks[3]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[3]), tuple(pose_landmarks[7]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[9]), tuple(pose_landmarks[10]), (255, 255, 255), 2)

    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[12]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[11]), tuple(pose_landmarks[23]), (255, 205, 0), 2)

    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[23]), (255, 255, 255), 2)
    cv2.line(image, tuple(pose_landmarks[24]), tuple(pose_landmarks[12]), (0, 144, 255), 2)

    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[16]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[14]), tuple(pose_landmarks[12]), (0, 144, 255), 2)

    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[22]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[20]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[16]), tuple(pose_landmarks[18]), (0, 144, 255), 2)
    cv2.line(image, tuple(pose_landmarks[18]), tuple(pose_landmarks[20]), (0, 144, 255), 2)

    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[13]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[21]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[19]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[15]), tuple(pose_landmarks[17]), (255, 205, 0), 2)
    cv2.line(image, tuple(pose_landmarks[19]), tuple(pose_landmarks[17]), (255, 205, 0), 2)


    for lm in pose_landmarks:

        cv2.circle(image, (int(lm[0]), int(lm[1])), 5, (255,255,255), 4)
        cv2.circle(image, (int(lm[0]), int(lm[1])), 4, (255,255,141), -1)

def show_fps(image, prev_frame_time):
    new_frame_time = time.time()
    fps = int(1/(new_frame_time-prev_frame_time))
    cv2.putText(image, f"fps: {fps}", (1000, 700), cv2.FONT_HERSHEY_SIMPLEX, 2, (100, 255, 0), 2, cv2.LINE_AA)
    return new_frame_time

def draw_class_on_image(label, img):
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, 30)
    fontScale = 1
    fontColor = (0, 255, 0)
    thickness = 2
    lineType = 2
    cv2.putText(img, label,
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                thickness,
                lineType)
    return img


def detect(model, lm_list):
    global label
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    print(lm_list.shape)
    results = model.predict(lm_list)
    print(results)
    if results[0][0] > 0.5:
        label = "cheat"
    else:
        label = "non_cheat"
    return label

def body_pose_detect(frame1, i, prev_frame_time, model,lm_list):
    image, results = mediapipe_detection(frame1, pose)
    
    frame1 = cv2.resize(frame1,(1280,720))

    i = i+1
    if i > warmup_frames:
        print("Start detect....")

        if results.pose_landmarks:
                draw_landmark_on_image( results, frame1)
                lm = get_frame_landmarks(results)
                lm_list.append(lm)
                print(lm)


                if len(lm_list) == n_time_steps:
                    t1 = threading.Thread(target=detect, args=(model, lm_list,))
                    t1.start()
                    lm_list = []

                print(lm)

    frame1 = draw_class_on_image(label, frame1)
    prev_frame_time = show_fps(frame1, prev_frame_time)
    
    return frame1
        


# head pose & face detect

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
mouth_movements = []
iw, ih =  720, 580
warn_mouth = ""

cap = cv2.VideoCapture(0)

def get_mouth_movement(upper_mouth, bottom_mouth):
    distance = abs(int(upper_mouth.y * ih) - int(bottom_mouth.y * ih))
    if distance > 8:
        return 1
    else:
        return 0

def warning(warnings):
    count = 0
    for warning in warnings:
        if warning == 1:
            count = count + 1;
    if count >= 5:
        return "warning: Talking"
    else : 
        return ""



def face_detect(image, results):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The distortion parameters
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
          

            if y < -10:
                text = "Looking Left"
            elif y > 10:
                text = "Looking Right"
            elif x < -10:
                text = "Looking Down"
            elif x > 10:
                text = "Looking Up"
            else:
                text = "Forward"

            cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                                cv2.LINE_AA)
            cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            
    return image



In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture(0)


with mp_face_mesh.FaceMesh(
    
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        
        if not success:
            print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
            continue
            
        image = cv2.resize(image,(720,580))

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:

                lmks = face_landmarks.landmark

                mouth_movement = get_mouth_movement(lmks[13], lmks[14])
                mouth_movements.append(mouth_movement)
                X = (int(lmks[113].x*image.shape[1]),int(lmks[113].y*image.shape[0]))
                Y = (int(lmks[434].x*image.shape[1]),int(lmks[434].y*image.shape[0]))
#                 print(image.shape)
                print(X)
                print(Y)
                cv2.rectangle(image, Y, X,(179,222,245),-1)
                if len(mouth_movements) == 16:
                    warn_mouth = warning(mouth_movements)
                    mouth_movements = []
                    

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
            
        # Flip the image horizontally for a selfie-view display.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        if warn_mouth != "":
            image = cv2.putText(image, str(warn_mouth), (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2,cv2.LINE_AA)
        image = face_detect(image , results)

#         cv2.imshow('Head Pose Estimation', image)

        cv2.imshow('MediaPipe Face Mesh',image)


        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()